<a href="https://colab.research.google.com/github/Hokaid/Machine-learning-for-COVID19-and-weather/blob/master/CC57_EB_201716913_201711814.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**[Enlace del notebook en linea](https://colab.research.google.com/drive/1PsrmL6xA_2tIGs66IOsZMsQ2lfyAFtWT?usp=sharing)**

#***VIDEOS DE EXPLICACIÓN***

##***Video en el que se explica las tecnicas de procesamiento de datos y feature selection (Exposición: GERAL ESTEEN CASTILLO ARREDONDO):*** **[Enlace para el video](https://www.loom.com/share/df993f1a7b334cb3a38aae85b4a0deec)**

##***Video en el que se explica el desarrollo de workflows (Exposición: JAVIER ROZAS):*** **[Enlace para el video](https://www.loom.com/share/48c53824c2924d19a9c4e77306623c03)**

#***Índice***

***I. Lectura y Descripción de los datos***

   1. ***Lectura de datos***

   2. ***Descripción del conjunto de datos***

***II. Procesamiento de datos***

   1. ***Limpieza de datos***

      * ***Valores duplicados***

      * ***Detección de outliers***

      * ***Detección de inconsistencia de datos***

      * ***Detección de valores faltantes***

   2. ***Transformación de datos***

   3. ***Reducción de datos***

   4. ***Discretización de datos***

   5. ***Feature construction***

   6. ***Tabla o reporte final***

***III. Feature Selection***

   1. ***Filtering***

      * ***Aplicación del Método de filtrado con correlación de Pearson***

   2. ***Embedded***

      * ***Aplicación del Método embebido***

   3. ***Wrapper***

      * ***Aplicación del Método Wrapper: Eliminación de Característica Recursiva***

   4. ***PCA***

      * ***Aplicación de Análisis de componentes principales (PCA)***

***IV. Workflows***

   1. ***Introducción y contexto***

   2. ***Listado de Workflows***

   3. ***Comparación y conclusión***

#***I. Lectura y Descripción de los datos***

##***1. Lectura de datos***

En las siguientes 2 celdas de codigo, se lee el conjunto de datos respectivo desde google drive. El archivo correspondiente a leer posee como nombre ***tempdata.csv***. 

In [ ]:
#Leer un CSV 
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials 
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
import pandas as pd
import numpy as np
from google.colab import data_table

link_google_drive = 'https://drive.google.com/open?id=11z8TZ71D5lH5VUACkPQryM1nhLu1wPSO'
flu, id = link_google_drive.split('=')
dataset = drive.CreateFile({'id':id})
dataset.GetContentFile('tempdata.csv')
df = pd.read_csv('tempdata.csv')
df.head()

,province,country,lat,long,date,cases,fatalities,capital,humidity,sunHour,tempC,windspeedKmph
0,NaN,Afghanistan,33.0,65.0,22/01/2020,0,0,Kabul,65.0,8.7,-1.0,8.0
1,NaN,Afghanistan,33.0,65.0,23/01/2020,0,0,Kabul,59.0,8.7,-3.0,8.0
2,NaN,Afghanistan,33.0,65.0,24/01/2020,0,0,Kabul,71.0,7.1,0.0,7.0
3,NaN,Afghanistan,33.0,65.0,25/01/2020,0,0,Kabul,79.0,8.7,0.0,7.0
4,NaN,Afghanistan,33.0,65.0,26/01/2020,0,0,Kabul,64.0,8.7,-1.0,8.0


##***2. Descripción del conjunto de datos***

***Contexto del dataset:*** Este conjunto de datos fue hecho con el objetivo de cuantificar y analizar la correlación entre los ***datos meteorológicos básicos*** y la transmisión del virus ***COVID-19***. Entre los datos, se encuentran temperatura promedio, horas de luz solar, porcentaje de humedad y velocidad del viento en la capital de 156 países. Las fechas varian del 22/1/2020 al 21/3/2020.

Ahora bien, se procedera a definir cada atributo del conjunto de datos obtenido y a identificar el atributo clase o objetivo:

***Caracteristicas***

* **Provincia (province):** indica la provincia, la cual se encuentra en la capital del país respectivo, en la que se reportaron los casos y muertos por coronavirus.

* **País (country):** indica el País en el que se reportaron los casos y muertos por coronavirus.

* **Latitud (lat):** indica la latitud correspondiente a la región en la que se reportaron los casos y muertos por coronavirus.

* **Longitud (long):** indica la longitud correspondiente a la región en la que se reportaron los casos y muertos por coronavirus.

* **Fecha (date):** indica la fecha en la que se reportaron los casos, recuperados y muertos por coronavirus.

* **Muertos (fatalities):** indica la cantidad de muertos de coronavirus respectiva.

* **Capital (capital):** indica la capital, la cual se encuentra en el país respectivo, en la que se reportaron los casos y muertos por coronavirus.

* **Humedad (humidity):** indica el porcentaje de humedad correspondiente a la región en la que se reportaron los casos y muertos por coronavirus.

* **Horas de luz solar (sunHour):** indica la horas de luz solar correspondiente a la región en la que se reportaron los casos y muertos por coronavirus

* **Temperatura (tempC):** indica la temperatura promedio en Celcius correspondiente a la región en la que se reportaron los casos y muertos por coronavirus

* **Velocidad del viento (windspeedKmph):** indica velocidad del viento correspondiente a la región en la que se reportaron los casos y muertos por coronavirus

***Atributo clase o objetivo***

* ***Casos (cases):*** indica la cantidad de casos de coronavirus respectiva. Se eligio este atributo como columna clase, ya que con respecto a los otros datos tu lo puedes saber o determinar con facilidad. Si alguien esta muerto, es facil de identificar. No obstante, con respecto a **Casos (cases)**, es necesario realizar pruebas de descarte para identificar los casos de coronavirus respectivo. Los modelos que se presentaran a continuación, permitiran predecir la cantidad de casos de coronavirus sin necesidad de pruebas medicas. En ese sentido, representa una ventaja, ya que dichas pruebas son de dificil acceso para ciertas regiones. 

#***II. Procesamiento de Datos***

##***1. Limpieza de datos***

###***Valores duplicados***

El siguiente codigo es capaz de identificar y contar la cantidad de registros duplicados en el conjunto de datos analizado. Se impreme el valor de 0, por lo que se llega a una conclusión.

***Conslución: No existen registros duplicados en el conjunto de datos***

In [ ]:
#identificar si hay valores duplicados
ddf = df.duplicated()
n_vdupli = 0 #numeros de valores duplicados
for i in range(len(ddf)):
  if (ddf[i] == True):
    n_vdupli += 1
print(n_vdupli)

0


###***Detección de outliers***

Ahora bien, se procedera a identificar la posible precencia de outliers en cada atributo numerico del conjunto de datos:

* ***Latitud (lat)***

![alt text](https://i.ibb.co/Z80Mpsc/bp3.png)

Se debe considerar el  Rango: [-41.45; 71.71]

* ***Longitud (long)***

![alt text](https://i.ibb.co/vzSZDmt/bp4.png)

Se debe considerar el  Rango: [-157.50; 174.89]

* ***Casos (cases)***

![alt text](https://i.ibb.co/gW3qshL/bp6.png)

Se debe considerar el  Rango: [0; 67800]

* ***Muertes (fatalities)***

![alt text](https://i.ibb.co/MBzsCcX/bp7.png)

Se debe considerar el  Rango: [0; 4825]

* ***Humedad (humidity)***

![alt text](https://i.ibb.co/hc08c3D/bp8.png)

Se debe considerar el  Rango: [5; 99]

* ***Horas de luz solar (sunHour)***

![alt text](https://i.ibb.co/99btbc7/bpsolar.png)

Se debe considerar el  Rango: [1.5; 14]

* ***Temperatura en grados Celsius (tempC)***

![alt text](https://i.ibb.co/0nKn7Sr/bptemp.png)

Se debe considerar el  Rango: [-21; 45]

* ***Velocidad del viento en km/h (windspeedKmph)***

![alt text](https://i.ibb.co/nL8QP0k/bpspeed.png)

Se debe considerar el  Rango: [1; 71]

***Conclusión: Como se puede apreciar en los diagramas de caja dados anteriormente, no existen valores que superen los extremos superiores e inferiores en ninguno de ellos. Por lo tanto, se concluye que no existen outliers o valores atipicos para cada caso. En ese sentido, no existen outliers con respecto a atributos numericos en el conjunto de datos analizado.***





###***Detección de inconsistencia de datos***

Se utilizo la herramienta **Orange** para la determinación de valores inconsistentes o inconsistencia en los datos. A continuación, se muestran las estadisticas de los datos dados por la herramienta mencionada:

![alt text](https://i.ibb.co/VHfdd0S/1report.png)
![alt text](https://i.ibb.co/4SbStdf/2report.png)

De acuerdo a la información dada anteriormente, no se a logrado identificar ningun tipo de inconsistencia en ningun atributo con respecto a el rango, la media, dispersión y otros valores estadisticos analizados. 

***Conclusión: No hay una inconsistencia significativa en los datos***

###***Detección de valores faltantes***

A continuación, se imprimen los valores faltantes por cada columna. En este caso, podemos identificar que existe una gran cantidad de valores faltantes en el conjunto de datos. 

In [ ]:
 print("cantidad de registros: ", len(df))
 df.isnull().sum()

cantidad de registros:  16677


province         8937
country             0
lat                 0
long                0
date                0
cases               0
fatalities          0
capital             0
humidity          177
sunHour           177
tempC             177
windspeedKmph     177
dtype: int64

Se logran encontrar valores faltantes en los siguientes atributos: ***Province (8937), Humidity (177), suHour (177), tempC (177) y windspeedKmph (177)***. Se plantean 2 decisiones de experimentación para eliminar los valores faltantes. Estas se describen a continuación:

* ***Primera decisión: Consiste en remplazar los valores faltantes por la media o valor central dependiendo de la naturaleza del atributo.*** La implementación de la misma, se muestra a continuación. Luego de aplicarse la tecnica implementada sobre el conjunto de datos, se imprimen la cantidad de valores faltantes por atributo y los 5 primeros registros del conjunto de datos resultante. 

In [ ]:
#Fución que define la primera tecnica de limpieza de datos propuesta
def natocenter(df):
  names = df.columns.values
  for i in range(len(names)): 
    if df.dtypes[i] == 'object':
      df = df.fillna({names[i]: df.describe(include = 'all').iloc[2][i]})
    elif df.dtypes[i] == 'int64' or df.dtypes[i] == 'float64':
      df = df.fillna({names[i]: df.describe(include = 'all').iloc[4][i]})
  return df

dfsinna = natocenter(df)
print(dfsinna.isnull().sum())
dfsinna.head()

province         0
country          0
lat              0
long             0
date             0
cases            0
fatalities       0
capital          0
humidity         0
sunHour          0
tempC            0
windspeedKmph    0
dtype: int64


,province,country,lat,long,date,cases,fatalities,capital,humidity,sunHour,tempC,windspeedKmph
0,Grand Princess,Afghanistan,33.0,65.0,22/01/2020,0,0,Kabul,65.0,8.7,-1.0,8.0
1,Grand Princess,Afghanistan,33.0,65.0,23/01/2020,0,0,Kabul,59.0,8.7,-3.0,8.0
2,Grand Princess,Afghanistan,33.0,65.0,24/01/2020,0,0,Kabul,71.0,7.1,0.0,7.0
3,Grand Princess,Afghanistan,33.0,65.0,25/01/2020,0,0,Kabul,79.0,8.7,0.0,7.0
4,Grand Princess,Afghanistan,33.0,65.0,26/01/2020,0,0,Kabul,64.0,8.7,-1.0,8.0


* ***Segunda decisión: Consiste en remplazar los valores faltantes por el valor de 0.*** La implementación de la misma, se muestra a continuación. Luego de aplicarse la tecnica implementada sobre el conjunto de datos, se imprimen la cantidad de valores faltantes por atributo y los 5 primeros registrso del conjunto de datos resultante.

In [ ]:
#Fución que define la segunda tecnica de limpieza de datos propuesta
def natozero(df):
  names = df.columns.values
  for i in range(len(names)): 
    if df.dtypes[i] == 'object':
      df = df.fillna({names[i]: '0'})
    elif df.dtypes[i] == 'int64' or df.dtypes[i] == 'float64':
      df = df.fillna({names[i]: 0})
  return df

dfsinna = natozero(df)
print(dfsinna.isnull().sum())
dfsinna.head()

province         0
country          0
lat              0
long             0
date             0
cases            0
fatalities       0
capital          0
humidity         0
sunHour          0
tempC            0
windspeedKmph    0
dtype: int64


,province,country,lat,long,date,cases,fatalities,capital,humidity,sunHour,tempC,windspeedKmph
0,0,Afghanistan,33.0,65.0,22/01/2020,0,0,Kabul,65.0,8.7,-1.0,8.0
1,0,Afghanistan,33.0,65.0,23/01/2020,0,0,Kabul,59.0,8.7,-3.0,8.0
2,0,Afghanistan,33.0,65.0,24/01/2020,0,0,Kabul,71.0,7.1,0.0,7.0
3,0,Afghanistan,33.0,65.0,25/01/2020,0,0,Kabul,79.0,8.7,0.0,7.0
4,0,Afghanistan,33.0,65.0,26/01/2020,0,0,Kabul,64.0,8.7,-1.0,8.0


##***2. Transformación de datos***

Las caracteristicas poseen diferentes rangos, por lo que sus datos deben ser llevados a una misma escala. Antes de realizar una operación de ***normalización o estandarización*** sobre el conjunto de datos, es importante definir que todos estos esten expresados como numeros. Asimismo, los modelos de aprendizaje automatico, generalmente, solo entienden numeros. En ese sentido, es importante codificar los atributos categoricos, de tal manera que puedan ser representados por numeros. Luego de ello, se ejecuta la normalización o estandarización sobre todas las columnas, excepto aquella correspondiente a la columna clase o objetivo. ***El atributo objetivo debe ser indicado, ya que este no sera normalizado o estandarizado***. Se plantean 2 tecnicas de ***transformación de datos***. Estas se describen como sigue:

* ***Primera decisión: Consiste en transformar todos los datos a numeros. Luego de ello, normalizar todos los datos que no correspondan al atributo clase o objetivo.*** La implementación de la misma, se muestra a continuación. ***Esta parte del codigo no es del todo generica***, esto sucede, ya que se debe indicar el indice de la columna objetivo o clase. La columna objetivo o clase no posee el mismo indice en todos los conjuntos de datos. Luego de aplicar la tecnica, se imprime una descripción del conjunto de datos. ***Esta descripción muestra que todos los atributos, excepto el atributo clase o objetivo, poseen un rango de [0, 1].***

In [ ]:
from sklearn.preprocessing import LabelEncoder

def cod_normalize(df, clase):
  names = df.columns.values
  categorical_feature_mask = df.dtypes==object #Codificando todas las variables categoricas
  categorical_cols = df.columns[categorical_feature_mask].tolist()
  le = LabelEncoder()
  df[categorical_cols] = df[categorical_cols].apply(lambda col: le.fit_transform(col))
  for i in range(len(names)):
    if names[i] != clase:
      if (df[names[i]].max()-df[names[i]].min() != 0):
        df[names[i]] = (df[names[i]] - df[names[i]].min())/ (df[names[i]].max()-df[names[i]].min())
  return df

ind_clase = 5 #se debe indicar el indice de la columna objetivo, ya que esta no sera tratada
names = df.columns.values
dfn = cod_normalize(natocenter(df), names[ind_clase])
dfn.describe()

,province,country,lat,long,date,cases,fatalities,capital,humidity,sunHour,tempC,windspeedKmph
count,16677.000000,16677.000000,16677.000000,16677.000000,16677.000000,16677.000000,16677.000000,16677.000000,16677.000000,16677.000000,16677.000000,16677.000000
mean,0.361943,0.535974,0.599755,0.487186,0.500008,289.678120,0.002078,0.544985,0.654193,0.562382,0.561028,0.166084
std,0.235061,0.338742,0.202967,0.241242,0.293562,3354.089372,0.031005,0.332241,0.200850,0.200798,0.159493,0.108811
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.244094,0.193548,0.485127,0.258641,0.237288,0.000000,0.000000,0.222222,0.542553,0.392000,0.454545,0.085714
50%,0.244094,0.509677,0.657950,0.502431,0.508475,0.000000,0.000000,0.575163,0.712766,0.576000,0.545455,0.142857
75%,0.456693,0.948387,0.741956,0.669401,0.762712,9.000000,0.000000,0.921569,0.797872,0.760000,0.696970,0.214286
max,1.000000,1.000000,1.000000,1.000000,1.000000,67800.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


* ***Segunda decisión: Consiste en transformar todos los datos a numeros. Luego de ello, estandarizar todos los datos que no correspondan al atributo clase o objetivo.*** La implementación de la misma, se muestra a continuación. ***Esta parte del codigo no es del todo generica***, esto sucede, ya que se debe indicar el indice de la columna objetivo o clase. La columna objetivo o clase no posee el mismo indice en todos los conjuntos de datos. Luego de aplicar la tecnica, se imprime una descripción del conjunto de datos. ***Esta descripción muestra que todos los atributos, excepto el atributo clase o objetivo, poseen una media de aproximadamente 0.***

In [ ]:
from sklearn.preprocessing import LabelEncoder

def cod_standarize(df, clase):
  names = df.columns.values
  categorical_feature_mask = df.dtypes==object #Codificando todas las variables categoricas
  categorical_cols = df.columns[categorical_feature_mask].tolist()
  le = LabelEncoder()
  df[categorical_cols] = df[categorical_cols].apply(lambda col: le.fit_transform(col))
  names = df.columns.values
  for i in range(len(names)):
    if names[i] != clase:
      if (df[names[i]].std() != 0):
        df[names[i]] = (df[names[i]] - df[names[i]].mean())/ df[names[i]].std()
  return df

ind_clase = 7 #se debe indicar el indice de la columna objetivo, ya que esta no sera tratada
names = df.columns.values
dfe = cod_standarize(natocenter(df), names[ind_clase])
dfe.describe()

,province,country,lat,long,date,cases,fatalities,capital,humidity,sunHour,tempC,windspeedKmph
count,1.667700e+04,1.667700e+04,1.667700e+04,1.667700e+04,1.667700e+04,1.667700e+04,1.667700e+04,16677.000000,1.667700e+04,1.667700e+04,1.667700e+04,1.667700e+04
mean,8.856114e-15,-2.869982e-14,1.955063e-15,3.038943e-15,9.146125e-16,8.266670e-15,2.847574e-15,83.382683,-6.821218e-15,-1.029144e-13,-9.557694e-15,-1.395944e-15
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,50.832895,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.539782e+00,-1.582249e+00,-2.954942e+00,-2.019490e+00,-1.703245e+00,-8.636565e-02,-6.702064e-02,0.000000,-3.257128e+00,-2.800738e+00,-3.517570e+00,-1.526349e+00
25%,-5.013515e-01,-1.010875e+00,-5.647611e-01,-9.473672e-01,-8.949376e-01,-8.636565e-02,-6.702064e-02,34.000000,-5.558395e-01,-8.485256e-01,-6.676335e-01,-7.386157e-01
50%,-5.013515e-01,-7.763130e-02,2.867186e-01,6.319448e-02,2.884215e-02,-8.636565e-02,-6.702064e-02,88.000000,2.916236e-01,6.781917e-02,-9.764629e-02,-2.134599e-01
75%,4.030878e-01,1.217483e+00,7.006116e-01,7.553214e-01,8.948856e-01,-8.368236e-02,-6.702064e-02,141.000000,7.153552e-01,9.841640e-01,8.523324e-01,4.429848e-01
max,2.714433e+00,1.369849e+00,1.971971e+00,2.125727e+00,1.703193e+00,2.012776e+01,3.218534e+01,153.000000,1.721718e+00,2.179396e+00,2.752290e+00,7.663877e+00


##***3. Reducción de datos***

Llegados a este punto, se planteara la reducción de datos basandose en la cantidad de valores perdidos o nulos correspondientes del conjunto de datos analizado. A continuación, se imprimen la cantidad de registros y el porcentaje (%) de datos nulos por columna respectivamente.

In [ ]:
print("cantidad de registros: ", len(df))
print("Mostrar porcentaje (%) de datos faltantes por cada columna: ")
print((df.isnull().sum()/len(df))*100)

cantidad de registros:  16677
Mostrar porcentaje (%) de datos faltantes por cada columna: 
province         53.588775
country           0.000000
lat               0.000000
long              0.000000
date              0.000000
cases             0.000000
fatalities        0.000000
capital           0.000000
humidity          1.061342
sunHour           1.061342
tempC             1.061342
windspeedKmph     1.061342
dtype: float64


Dados los datos anteriores, se argumenta que existen datos faltantes en los siguientes atributos. Asimismo, se muestra el porcentaje de valores nulos asociado: 

* ***Provincia (province) : 53.6%***
* ***Humedad (humidity) : 1.1%***
* ***Horas de luz solar (sunHour) : 1.1%*** 
* ***Temperatura en grados Celcius (tempC) : 1.1%***
* ***Velocidad del viento en km/h (windspeedKmph) : 1.1%***

Como se puede observar, existe un porcentaje muy significativo de valores faltantes en la columna ***Provincia (province)***, ya que representa un ***53.6%*** de perdida. Por esta razón, se plantea la ***Eliminación de la columna Provincia (province)***. Esta decisión, se argumenta en el hecho de que menos del ***50%*** de los datos en dicha columna estan registrados. 

Por otro lado, el porcentaje de nulos para ***Humedad (humidity), Horas de luz solar (sunHour), Temperatura en grados Celcius (tempC) y Velocidad del viento en km/h (windspeedKmph)*** es practicamente el mismo. En ese sentido, se sospecha que los valores nulos en las 4 columnas mencionadas correspondan a las mismas filas o instancias del conjunto de datos. En primera instancia, es necesario identificar la cantidad de valores nulos en las 4 columnas detallas. En el siguiente codigo, se imprime la cantidad de valores nulos por columna. 

In [ ]:
print("Cantidad de valores nulos en las columnas mencionadas: ")
df.isnull().sum()

Cantidad de valores nulos en las columnas mencionadas: 


province         8937
country             0
lat                 0
long                0
date                0
cases               0
fatalities          0
capital             0
humidity          177
sunHour           177
tempC             177
windspeedKmph     177
dtype: int64

Ahora bien, es factible concluir que existen exactamente 177 valores nulos en cada una de las columnas ***Humedad (humidity), Horas de luz solar (sunHour), Temperatura en grados Celcius (tempC) y Velocidad del viento en km/h (windspeedKmph)***. Se requiere comprobar si estos valores nulos corresponden a las mismas instancias en el conjunto de datos. ***En otras palabras, saber si existen 177 registros que cuenten con datos faltantes en las 4 columnas especificadas***. Para ello, se realizara la respectiva verificación y conteo. El siguiente codigo permite contar ***todos los registros que poseen valores nulos en todas las 4 columnas mencionadas***. El resultado de dicho conteo, se imprime respectivamente. 

In [ ]:
arrna = pd.isnull(df).values
contna_cols = 0
for i in range(len(arrna)):
  contna_rows = 0
  for j in range(len(arrna[i])):
    if (arrna[i][j]): contna_rows += 1
  if (contna_rows >= 4): contna_cols += 1
print(contna_cols)

177


El resultado del conteo anterior, es efectivamente ***177***. Con ello, se comprueba que ***existen 177 registros que cuenten con datos faltantes en las columnas Humedad (humidity), Horas de luz solar (sunHour), Temperatura en grados Celcius (tempC) y Velocidad del viento en km/h (windspeedKmph)***. Ello implica aproximadamente un ***30%*** de valores faltantes por fila o registro. En ese sentido, se plantea la eliminación de aquellos registros que poseean un ***30%*** de valores faltantes o más. Con respecto a esta sección, se ha planteado la ***Eliminación de la columna Provincia (province)*** y ***la eliminación de los registros especificados***. Dado dicha información, se derivan 2 decisiones de experimentación correspondiente a ***Reducción de datos*** para esta parte:

* ***Primera decisión: Consiste en eliminar las columnas que poseean un 50% o más de datos faltantes y los registros que poseean un 30% o más de valores faltantes.*** La implementación de la misma, se muestra a continuación. Con respecto a la tecnica, se debe indicar el porcentaje minimo de datos nulos en una columna para poder eliminarla y el porcentaje minimo de datos nulos en una fila para poder eliminarla respectivamente. Dichos porcentajes deben indicarse como parametros de la función correspondiente a la implementación de la tecnica. Luego de aplicar la tecnica, se imprime la cantidad de valores nulos por cada columna restante. ***Se comprueba la eliminación total de nulos en el conjunto de datos***. 

In [ ]:
def reducedata(df, pcol, prow): #Implementación de la tecnica
  names = df.columns.values
  arrna = pd.isnull(df).values
  for j in range(len(arrna[0])):
    contna = 0
    for i in range(len(arrna)):
      if (arrna[i][j]): contna += 1
    if (contna/len(arrna) >= pcol): df = df.drop([names[j]], axis=1)
  for i in range(len(arrna)):
    contna = 0
    for j in range(len(arrna[i])):
      if (arrna[i][j]): contna += 1
    if (contna/len(arrna[i]) >= prow): df = df.drop([i],axis=0)
  return df

reducedf = reducedata(df, 0.5, 0.3) #Se deben indicar los porcentajes minimos de valores nulos para eliminar columnas y filas respectivamente. 
reducedf.isnull().sum()

country          0
lat              0
long             0
date             0
cases            0
fatalities       0
capital          0
humidity         0
sunHour          0
tempC            0
windspeedKmph    0
dtype: int64

* ***Segunda decisión: Consiste en no aplicar ningun tipo de reducción de datos***. Esta decisión se plantea como la antitesis de la tecnica vista anteriormente. ***Se plantea ello de esta manera, ya que se desea comparar el impacto de la aplicación y no aplicación de la reducción de datos sobre el desempeño de los clasificadores a probar***. Al no aplicarse ningun tipo de reducción, no es necesario la implementación de alguna tecnica.

##***4. Discretización de datos***

Ahora bien, con respecto a la discretización. ***El atributo clase o objetivo, identificado en el conjunto de datos, es de tipo numerico. Se debe discretizar dicha variable con el objetivo de facilitar el trabajo con clasificadores***. El atributo clase o objetivo corresponde a aquel que se va a predecir, por lo tanto su discretización facilita el proceso de predicción. Este atributo corresponde a la columna ***Casos (cases)***. ***Es importante considerar que los intervalos resultantes, luego de la discretización, deben contender la cantidad de datos más parecida posible entre ellos***. En ese sentido, se plantean las siguientes decisiones: 

* ***Primera decisión: Se trata de discretizar la variable clase o objetivo Casos (cases) en 2 intervalos***. La implementación de la tecnica respectiva se detalla a continuación. ***Esta parte no es del todo generica, ya que se debe de indicar el indice de la columna clase o objetivo***. Se debe indicar dicho indice, ya que la discretización solo se aplicara sobre la columna ***Casos (cases)***. Cabe mencionar que se esta priorizando que ambos intervalos poseean la cantidad de datos lo más parecida posible. Luego de aplicada la tecnica, se imprime la cantidad de datos por cada clase o intervalo generado y codificado como ***0 o 1***. 

In [ ]:
def discre2(dframe, col): #Implementación de la tecnica 
  dframe[col] = pd.qcut(dframe[col], 3, duplicates = 'drop')
  for i in range(len(dframe[col].unique())): 
    dframe[col] = dframe[col].replace([dframe[col].unique()[i]], i)
  dframe[col] = dframe[col].astype(int)
  return dframe

ind_col = 5 #Se debe indicar el indice de la columna a discretizar
names = df.columns.values
dfd2 = df.copy()
dfd2 = discre2(dfd2, names[ind_col])
dfd2[names[ind_col]].value_counts()

0    11158
1     5519
Name: cases, dtype: int64

* ***Segunda decisión: Se trata de discretizar la variable clase o objetivo Casos (cases) en 3 intervalos***. La implementación de la tecnica respectiva se detalla a continuación. ***Esta parte no es del todo generica, ya que se debe de indicar el indice de la columna clase o objetivo***. Se debe indicar dicho indice, ya que la discretización solo se aplicara sobre la columna ***Casos (cases)***. Cabe mencionar que se esta priorizando que los 3 intervalos poseean la cantidad de datos lo más parecida posible. Luego de aplicada la tecnica, se imprime la cantidad de datos por cada clase o intervalo generado y codificado como ***0, 1 o 2***.

In [ ]:
def discre3(dframe, col): #Implementación de la tecnica 
  dframe[col] = pd.qcut(dframe[col], 6, duplicates = 'drop')
  for i in range(len(dframe[col].unique())): 
    dframe[col] = dframe[col].replace([dframe[col].unique()[i]], i)
  dframe[col] = dframe[col].astype(int)
  return dframe

ind_col = 5 #Se debe indicar el indice de la columna a discretizar
names = df.columns.values
dfd3 = df.copy()
dfd3 = discre3(dfd3, names[ind_col])
dfd3[names[ind_col]].value_counts()

0    11158
2     2770
1     2749
Name: cases, dtype: int64

##***5. Feature Construction***

Continuando con el asunto, en el conjunto de datos analizado, se ha detectado una atributo ***Fecha (date)***. Debido a la naturaleza de este atributo, no es posible trabajar el mismo como una variable categorica. Por lo tanto, se plantea construir, a partir de esta caracteristica, 3 caracteristicas que representen lo mismo. En otras palabras, remplazar el atributo ***Fecha (date)*** por caracteristicas como ***Dia (day), Mes (month) y Año (year)***. Con respecto a ello, se plantean 2 decisiones de experimentación. En la primera, se realizara lo mencionado. Por otro lado, en la segunda decisión, no se construira ninguna nueva caracteristica. Esto ultimo con el objetivo de comprobar el impacto de ***Feature Construction***. 

* ***Primera decisión: Remplazar la caracteristica Fecha (date) por tres nuevas caracteristicas correspondientes a Dia (day), Mes (month) y Año (year) respectivamente.*** La implementación de la tecnica respectiva se detalla a continuación. ***Esta parte no es del todo generica, ya que se debe de indicar el indice de la columna a tratar***. En este caso, se debe indicar el indice de la caracteristica ***Fecha (date)***. Luego de aplicada la tecnica, se imprimen los 5 primeros registros del conjunto de datos. Asimismo, se evidencia el remplazo de la caracteristica ***Fecha (date)*** por las caracteristicas ***Dia (day), Mes (month) y Año (year) respectivamente***.

In [ ]:
def datetodmy(df, date): #implementación de la técnica
  df[date] =  pd.to_datetime(df[date])
  df["day"] = df[date].map(lambda x: x.day)
  df["month"] = df[date].map(lambda x: x.month)
  df["year"] = df[date].map(lambda x: x.year)
  df = df.drop([date], axis=1)
  return df

names = df.columns.values
ind_date = 4 #Se debe indicar el indice de la columna a tratar
dfdmy = df.copy()
dfdmy = datetodmy(dfdmy, names[ind_date])
dfdmy.head()

,province,country,lat,long,cases,fatalities,capital,humidity,sunHour,tempC,windspeedKmph,day,month,year
0,NaN,Afghanistan,33.0,65.0,0,0,Kabul,65.0,8.7,-1.0,8.0,22,1,2020
1,NaN,Afghanistan,33.0,65.0,0,0,Kabul,59.0,8.7,-3.0,8.0,23,1,2020
2,NaN,Afghanistan,33.0,65.0,0,0,Kabul,71.0,7.1,0.0,7.0,24,1,2020
3,NaN,Afghanistan,33.0,65.0,0,0,Kabul,79.0,8.7,0.0,7.0,25,1,2020
4,NaN,Afghanistan,33.0,65.0,0,0,Kabul,64.0,8.7,-1.0,8.0,26,1,2020


* ***Segunda decisión: Se trata de no construir ninguna nueva caracteristica y dejar el conjunto de datos tal cual***. Esta decisión se plantea como la antitesis de la tecnica vista anteriormente. ***Se plantea ello de esta manera, ya que se desea comparar el impacto de la aplicación y no aplicación de Feature Construction sobre el desempeño de los clasificadores a probar***. Al no aplicarse ningun tipo de ***Feature Construction***, no es necesario la implementación de alguna técnica. 

##***6. Tabla o reporte final***

Ahora bien, se debe dar un ***reporte*** como resumen final de lo aplicado en esta parte. A continuación, se encuentra la tabla correspondiente:

![alt text](https://i.ibb.co/88h7tmj/tabla-reporte-eb.png)

Esta tabla corresponde a la tabla dada en el enunciado del examen respectivo. Asimismo, en dicha tabla se reportan todas las tecnicas o decisiones a aplicar y combinar en los ***Workflows*** de la parte 3. ***Cada tecnica o decisión fue correctamente sustentada en los apartados anteriores*** y posee una nomenclatura respectiva a la actividad realizada. En ese sentido, las tecnicas de ***limpieza de datos*** se identifican como ***L1*** y ***L2***. Analogicamente, el mismo proceso se cumple para el resto de actividades. 

#***III. Feature Selection***

## ***1. Filtering***

### ***Aplicación de Método de filtrado con correlación de Pearson***

En el siguiente codigo, se esta implementando una técnica de ***Filtering***. Dicha tecnica corresponde a la aplicación de el ***Método de filtrado con correlación de Pearson***. ***Esta parte del codigo no es del todo generica, ya que se debe indicar el indice de la columna clase o objetivo***. La columna clase o objetivo debe ser especificada con el objetivo de que no entre en el proceso de ***selección de caracteristicas***. Luego de la aplicación de la técnica, se imprime el conjunto de datos resultante con las ***caracteristicas seleccionadas y el atributo objetivo***.  

In [ ]:
def filterpearson(df, clase): #Implementación de la técnica
  names = df.columns.values
  cor = df.corr()
  cor_target = abs(cor[clase])
  peacols = []
  for i in range(len(names)):
    if (names[i] != clase and cor_target[i] >= 0.1):
      peacols += [names[i]]
  df_x = df[peacols]
  df_y = df[clase]
  df = pd.concat([df_x,df_y],axis=1)
  return df

ind_clase = 5 #Se debe indicar el indice de la columna clase o objetivo
names = df.columns.values
dffp = filterpearson(cod_standarize(natocenter(df), names[ind_clase]), names[ind_clase])
dffp.head()

,fatalities,humidity,cases
0,-0.067021,-0.079141,0
1,-0.067021,-0.396940,0
2,-0.067021,0.238657,0
3,-0.067021,0.662389,0
4,-0.067021,-0.132108,0


## ***2. Embedded***

### ***Aplicación de Método embebido***

En el siguiente codigo, se esta implementando una técnica de ***Embedded***. Dicha tecnica corresponde a la aplicación de el ***Método de embebido***. ***Esta parte del codigo no es del todo generica, ya que se debe indicar el indice de la columna clase o objetivo***. La columna clase o objetivo debe ser especificada con el objetivo de que no entre en el proceso de ***selección de caracteristicas***. Luego de la aplicación de la técnica, se imprime el conjunto de datos resultante con las ***caracteristicas seleccionadas y el atributo objetivo***.

In [ ]:
from sklearn.linear_model import LassoCV

def embebido(df, clase): #Implementación de la técnica
  X = df.drop(clase, 1) 
  Y = df[clase]
  reg = LassoCV()
  reg.fit(X, Y)
  coef = pd.Series(reg.coef_, index = X.columns)
  embbcols = coef[abs(coef) > 0.1].index
  df_x = df[embbcols]
  df_y = df[clase]
  df = pd.concat([df_x,df_y],axis=1)
  return df

ind_clase = 5 #Se debe indicar el indice de la columna clase o objetivo
names = df.columns.values
emdf = embebido(cod_standarize(natocenter(df), names[ind_clase]), names[ind_clase])
emdf.head()

,province,lat,long,date,fatalities,capital,humidity,sunHour,tempC,windspeedKmph,cases
0,-0.501352,0.286719,0.755321,-0.086630,-0.067021,-0.459991,-0.079141,0.067819,-1.617612,-0.607327,0
1,-0.501352,0.286719,0.755321,0.028842,-0.067021,-0.459991,-0.396940,0.067819,-1.807608,-0.607327,0
2,-0.501352,0.286719,0.755321,0.144315,-0.067021,-0.459991,0.238657,-0.569638,-1.522614,-0.738616,0
3,-0.501352,0.286719,0.755321,0.259787,-0.067021,-0.459991,0.662389,0.067819,-1.522614,-0.738616,0
4,-0.501352,0.286719,0.755321,0.375260,-0.067021,-0.459991,-0.132108,0.067819,-1.617612,-0.607327,0


## ***3. Wrapper***

### ***Aplicación de Método Wrapper: Eliminación de Característica Recursiva***

En el siguiente codigo, se esta implementando una técnica de ***Wrapper***. Dicha tecnica corresponde a la aplicación de el ***Método Wrapper: Eliminación de Característica Recursiva***. ***Esta parte del codigo no es del todo generica, ya que se debe indicar el indice de la columna clase o objetivo***. La columna clase o objetivo debe ser especificada con el objetivo de que no entre en el proceso de ***selección de caracteristicas***. Luego de la aplicación de la técnica, se imprime el conjunto de datos resultante con las ***caracteristicas seleccionadas y el atributo objetivo***.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

def recursivewrapper(df, clase): #Implementación de la técnica
  X = df.drop(clase, 1) 
  Y = df[clase]
  high_score=0
  nof=0
  for i in range(len(X.columns)):
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.3, random_state = 0)
    model = LinearRegression()
    rfe = RFE(model,i+1)
    X_train_rfe = rfe.fit_transform(X_train,Y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,Y_train)
    score = model.score(X_test_rfe,Y_test)
    if(score>high_score):
      high_score = score
      nof = i+1
  model = LinearRegression()
  rfe = RFE(model, nof)
  X_rfe = rfe.fit_transform(X,Y)
  model.fit(X_rfe,Y)
  temp = pd.Series(rfe.support_,index = X.columns)
  rfecols = temp[temp==True].index
  df_x = df[rfecols]
  df_y = df[clase]
  df = pd.concat([df_x,df_y],axis=1)
  return df

ind_clase = 5 #Se debe indicar el indice de la columna clase o objetivo
names = df.columns.values
wradf = recursivewrapper(cod_standarize(natocenter(df), names[ind_clase]), names[ind_clase])
wradf.head()

,lat,long,date,fatalities,capital,humidity,tempC,cases
0,0.286719,0.755321,-0.086630,-0.067021,-0.459991,-0.079141,-1.617612,0
1,0.286719,0.755321,0.028842,-0.067021,-0.459991,-0.396940,-1.807608,0
2,0.286719,0.755321,0.144315,-0.067021,-0.459991,0.238657,-1.522614,0
3,0.286719,0.755321,0.259787,-0.067021,-0.459991,0.662389,-1.522614,0
4,0.286719,0.755321,0.375260,-0.067021,-0.459991,-0.132108,-1.617612,0


## ***4. PCA***

### ***Aplicación de Análisis de componentes principales (PCA)***

En el siguiente codigo, se esta implementando una técnica de ***PCA***. Dicha tecnica corresponde a la aplicación de el ***Análisis de componentes principales (PCA)***. La técnica de ***PCA*** aplicada no solo reduce la dimensionalidad de los datos, sino que busca el numero de ***componentes principales optimo***. En otras palabras, se busca el numero de ***componentes principales*** con los que se maximice el performance de los clasificadores. Ahora bien, para validar dicha performance, se usa la función ***dtree_score***. Dicha función retorna la precisión promedio aplicando ***Arboles de decisión*** y un muestreo de ***cross-validation estratificado con k=15***. ***Esta parte del codigo no es del todo generica, ya que se debe indicar el indice de la columna clase o objetivo***. La columna clase o objetivo debe ser especificada con el objetivo de que no entre en el proceso de ***reducción de dimensionalidad***. Luego de la aplicación de la técnica, se imprime el conjunto de datos resultante con las ***caracteristicas resultantes y el atributo objetivo***.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score

def dtree_score(X, Y): #Arbol de decisión y cross-validation con k=15
  dtree = DecisionTreeClassifier()
  return cross_val_score(dtree, X, Y, cv=15).mean()

def pca(df, clase): #Implementación de la técnica de pca
  names = df.columns.values
  y = df[clase].values
  x = StandardScaler().fit_transform(df.drop(clase, 1).values)
  score = 0
  optimo_n = 2
  for n in range(len(names)-1, 1, -1): #Buscando el valor optimo para el numero de componentes principales de PCA
    pca = PCA(n_components=n)
    if (score <= dtree_score(pca.fit_transform(x), df.iloc[:,ind_clase].values)):
      score = dtree_score(pca.fit_transform(x), df.iloc[:,ind_clase].values)
      optimo_n = n
  pca = PCA(n_components=optimo_n)
  df_x = pd.DataFrame(pca.fit_transform(x))
  df_y = df[clase]
  df = pd.concat([df_x,df_y],axis=1)
  return df

ind_clase = 5 #Se debe indicar el indice de la columna clase o objetivo
names = df.columns.values
pcadf = pca(discre2(cod_standarize(natocenter(df), ind_clase), names[ind_clase]), names[ind_clase])
pcadf.head()

,0,1,2,cases
0,0.555058,1.998843,0.219198,0
1,0.616851,2.118305,0.380303,0
2,0.244448,2.189261,-0.037226,0
3,0.295010,1.945180,0.000473,0
4,0.563083,2.035497,0.244809,0


#***IV. Workflows***

##***1. Introducción y contexto***

Con respecto a esta parte, se diseñaran workflows a partir de las tecnicas abordadas anteriormente. Se realizara una combinación de todas las tecnicas, generando ***128 Workflows***. Las ***tecnicas de preprocesamiento de datos*** están representadas por variables para facilitar su nomenclatura. A continuación, se presenta una tabla, la cual indica la actividad de proprocesamiento, las tecnicas respectivas y ***la variable que viene a representar a cada tecnica***. 

![alt text](https://i.ibb.co/KmTG2xW/workvars.png)

En ese sentido, las ***tecnicas de preprocesamiento*** estan correctamente representadas, de tal manera que sean facilmente rastreables cuando se desarrollen los ***workflows*** respectivos. Ahora bien, las tecnicas correspondientes a ***Feature Selection*** tambien seran usadas en la generación de ***workflows***. Los nombres de las técnicas en si son muy largos, por lo que se utilizará el nombre de la actividad respectiva para identificarlas. Esto se hace así por motivos de nomenclatura y rastreo. Se procede a presentar un resumen de la técnicas y actividades relacionadas a ***Feature Selection*** aplicadas. 

![alt text](https://i.ibb.co/gRGKcFm/fiselec.png)

Los ***Workflows desarrollados surgen de una serie de combinaciones de tecnicas y/o decisiones de preprocesamiento de datos***. Cada combinación da como resultado un conjunto de datos preprocesado de forma distinta. ***Los datos resultantes de cada Workflow seran probados con 3 clasificadores para verificar la variabilidad del mismo frente al dataset y el pre-procesamiento realizado***. Los clasificadores a usar, se listan a continuación:

* ***K Vecinos más Cercanos (KNN)***
* ***Arbolés de Decisión***
* ***Maquinas de Vectores de Soporte (SVM)***

En el siguiente codigo, se definen 2 funciones. La primera función ***selecionarEsquemaWorkflow*** tiene como objetivo permitir seleccionar una combinación de procesamiento de datos y tecnicas, dado un arreglo.  La ultima función ***prove*** posse el proposito de probar los conjuntos de datos, dado un esquema de preprocesamiento, con 3 clasificadores. Las dos funciones especificadas son utilizadas en el desarrollo de los ***128 Workflows*** determinados. Asimismo, ***esta parte del codigo no es del todo generica***, ya que se debe indicar la ***columna clase o objetivo*** debido a los tratamientos realizados. 

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

names = df.columns.values
clase = names[5] #Se debe de indicar el nombre de la columna clase o objetivo

def seleccionarEsquemaWorkflow(df, wf): #Función para operar los workflows
  if wf[0]== 1: 
    if wf[5] != 4:
      df = reducedata(df, 0.5, 0.3) #Seleccionar tecnida de REDUCCION
  if wf[1]== 1: #Seleccionar tecnica de LIMPIEZA de datos
    df = natocenter(df)
  elif wf[1]== 2:
    df = natozero(df)
  if wf[2]== 1: #Seleccionar tecnica de FEATURE CONSTRUCTION
    if wf[5] != 4:
      df = datetodmy(df, names[4]) 
  if wf[3]== 1: #Selecionar tecnica de TRANSFORMAZION
    df = cod_normalize(df, clase) 
  elif wf[3]== 2:
    df = cod_standarize(df, clase) 
  if wf[5] == 1: #FEATURE SELECTION
    df = filterpearson(df, clase)
  elif wf[5] == 2:
    df = embebido(df, clase)
  elif wf[5] == 3:
    df = recursivewrapper(df, clase)
  elif wf[5] == 4:
    if wf[4] == 1: 
      df = discre2(df, clase)
    elif wf[4] == 2:
      df = discre3(df, clase)
    df = pca(df, clase)
  if (wf[5] >=1 and wf[5] <= 3): #Discretizar
    if wf[4] == 1: 
      df = discre2(df, clase)
    elif wf[4] == 2:
      df = discre3(df, clase)
  return df

def prove(df, wf, nfols): #Función que desarrolla el preprocesamiento y prueba con clasificadores
  data = seleccionarEsquemaWorkflow(df, wf)
  X = data.iloc[:,:-1].values #Caracteristicas
  Y = data[clase].values #Columna objetivo
  skf = StratifiedKFold(n_splits=nfols)
  knn = KNN(n_neighbors=5)
  dtree = DecisionTreeClassifier()
  svm = SVC()
  models = [knn, dtree, svm]
  scores = [0,0,0]
  for train_index, test_index in skf.split(X, Y):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    for i in range(len(models)):
      models[i].fit(X_train, Y_train)
      predicciones = models[i].predict(X_test)
      scores[i] += accuracy_score(Y_test, predicciones)
  print("Puntuación del clasificaodr KNN: ", scores[0]/nfols)
  print("Puntuación del clasificaodr Arból de Decisión: ", scores[1]/nfols)
  print("Puntuación del clasificaodr SVM: ", scores[2]/nfols)

##***2. Listado de Workflows***

***Llegados a este punto, se listaran los Workflows, aplicando las tecnicas respectivas y mostrando los resultados generados***. Cabe mencionar que cada ***Workflow*** posee como nombre una representación de la combinación de tecnicas implicadas en el mismo. 

###***1. R1 -> L1 -> F1 -> T1 -> D1 -> FILTERING***

In [ ]:
prove(df.copy(), [1,1,1,1,1,1], 10)

Puntuación del clasificaodr KNN:  0.7123636363636363
Puntuación del clasificaodr Arból de Decisión:  0.7841818181818183
Puntuación del clasificaodr SVM:  0.7024242424242424


### ***2. R2 -> L1 -> F1 -> T1 -> D1 -> FILTERING***

In [ ]:
prove(df.copy(), [2,1,1,1,1,1], 10)

Puntuación del clasificaodr KNN:  0.6958040406307228
Puntuación del clasificaodr Arból de Decisión:  0.7856871431469101
Puntuación del clasificaodr SVM:  0.7035994599641222


###***3.	R1 -> L2 -> F1 -> T1 -> D1 -> FILTERING***

In [ ]:
prove(df.copy(), [1,2,1,1,1,1], 10)

Puntuación del clasificaodr KNN:  0.7123636363636363
Puntuación del clasificaodr Arból de Decisión:  0.7841818181818183
Puntuación del clasificaodr SVM:  0.7024242424242424


###***4.	R2 -> L2 -> F1 -> T1 -> D1 -> FILTERING***

In [ ]:
prove(df.copy(), [2,2,1,1,1,1], 10)

Puntuación del clasificaodr KNN:  0.76062870159781
Puntuación del clasificaodr Arból de Decisión:  0.7994776224611194
Puntuación del clasificaodr SVM:  0.7260247231129313


###***5.	R1 -> L1 -> F2 -> T1 -> D1 -> FILTERING***

In [ ]:
prove(df.copy(), [1,1,2,1,1,1], 10)

Puntuación del clasificaodr KNN:  0.7123636363636363
Puntuación del clasificaodr Arból de Decisión:  0.7841818181818183
Puntuación del clasificaodr SVM:  0.7024242424242424


###***6.	R2 -> L1 -> F2 -> T1 -> D1 -> FILTERING***

In [ ]:
prove(df.copy(), [2,1,2,1,1,1], 10)

Puntuación del clasificaodr KNN:  0.6958040406307228
Puntuación del clasificaodr Arból de Decisión:  0.7856871431469101
Puntuación del clasificaodr SVM:  0.7035994599641222


###***7.	R1 -> L2 -> F2 -> T1 -> D1 -> FILTERING***

In [ ]:
prove(df.copy(), [1,2,2,1,1,1], 10)

Puntuación del clasificaodr KNN:  0.7123636363636363
Puntuación del clasificaodr Arból de Decisión:  0.7841818181818183
Puntuación del clasificaodr SVM:  0.7024242424242424


###***8.	R2 -> L2 -> F2 -> T1 -> D1 -> FILTERING***

In [ ]:
prove(df.copy(), [2,2,2,1,1,1], 10)

Puntuación del clasificaodr KNN:  0.76062870159781
Puntuación del clasificaodr Arból de Decisión:  0.7994776224611194
Puntuación del clasificaodr SVM:  0.7260247231129313


###***9.	R1 -> L1 -> F1 -> T2 -> D1 -> FILTERING***

In [ ]:
prove(df.copy(), [1,1,1,2,1,1], 10)

Puntuación del clasificaodr KNN:  0.7166060606060606
Puntuación del clasificaodr Arból de Decisión:  0.7841818181818183
Puntuación del clasificaodr SVM:  0.7074545454545454


###***10.	R2 -> L1 -> F1 -> T2 -> D1 -> FILTERING***

In [ ]:
prove(df.copy(), [2,1,1,2,1,1], 10)

Puntuación del clasificaodr KNN:  0.6965233571990638
Puntuación del clasificaodr Arból de Decisión:  0.7856871431469101
Puntuación del clasificaodr SVM:  0.7089973012591726


###***11.	R1 -> L2 -> F1 -> T2 -> D1 -> FILTERING***

In [ ]:
prove(df.copy(), [2,2,1,2,1,1], 10)

Puntuación del clasificaodr KNN:  0.76062870159781
Puntuación del clasificaodr Arból de Decisión:  0.7994776224611194
Puntuación del clasificaodr SVM:  0.7260247231129313


###***12. R2 -> L2 -> F1 -> T2 -> D1 -> FILTERING***

In [ ]:
prove(df.copy(), [2,2,2,2,1,1], 10)

Puntuación del clasificaodr KNN:  0.76062870159781
Puntuación del clasificaodr Arból de Decisión:  0.7994776224611194
Puntuación del clasificaodr SVM:  0.7260247231129313


###***13.	R1 -> L1 -> F2 -> T2 -> D1 -> FILTERING***

In [ ]:
prove(df.copy(), [1,1,2,2,1,1], 10)

Puntuación del clasificaodr KNN:  0.7166060606060606
Puntuación del clasificaodr Arból de Decisión:  0.7841818181818183
Puntuación del clasificaodr SVM:  0.7074545454545454


###***14.	R2 -> L1 -> F2 -> T2 -> D1 -> FILTERING***

In [ ]:
prove(df.copy(), [2,1,2,2,1,1], 10)

Puntuación del clasificaodr KNN:  0.6965233571990638
Puntuación del clasificaodr Arból de Decisión:  0.7856871431469101
Puntuación del clasificaodr SVM:  0.7089973012591726


###***15.	R1 -> L2 -> F2 -> T2 -> D1 -> FILTERING***

In [ ]:
prove(df.copy(), [1,2,2,2,1,1], 10)

Puntuación del clasificaodr KNN:  0.7166060606060606
Puntuación del clasificaodr Arból de Decisión:  0.7841818181818183
Puntuación del clasificaodr SVM:  0.7074545454545454


###***16.	R2 -> L2 -> F2 -> T2 -> D1 -> FILTERING***

In [ ]:
prove(df.copy(), [2,2,2,2,1,1], 10)

Puntuación del clasificaodr KNN:  0.76062870159781
Puntuación del clasificaodr Arból de Decisión:  0.7994776224611194
Puntuación del clasificaodr SVM:  0.7260247231129313


###***17.	R1 -> L1 -> F1 -> T1 -> D2 -> FILTERING***

In [ ]:
prove(df.copy(), [1,1,1,1,2,1], 10)

Puntuación del clasificaodr KNN:  0.7168484848484848
Puntuación del clasificaodr Arból de Decisión:  0.7677575757575756
Puntuación del clasificaodr SVM:  0.6896363636363637


###***18.	R2 -> L1 -> F1 -> T1 -> D2 -> FILTERING***

In [ ]:
prove(df.copy(), [2,1,1,1,2,1], 10)

Puntuación del clasificaodr KNN:  0.7111637025112962
Puntuación del clasificaodr Arból de Decisión:  0.769081795151761
Puntuación del clasificaodr SVM:  0.6916090882542916


###***19.	R1 -> L2 -> F1 -> T1 -> D2 -> FILTERING***

In [ ]:
prove(df.copy(), [1,2,1,1,2,1], 10)

Puntuación del clasificaodr KNN:  0.7168484848484848
Puntuación del clasificaodr Arból de Decisión:  0.7677575757575756
Puntuación del clasificaodr SVM:  0.6896363636363637


###***20.	R2 -> L2 -> F1 -> T1 -> D2 -> FILTERING***

In [ ]:
prove(df.copy(), [2,2,1,1,2,1], 10)

Puntuación del clasificaodr KNN:  0.7816734494827653
Puntuación del clasificaodr Arból de Decisión:  0.7816734494827653
Puntuación del clasificaodr SVM:  0.7173365686574915


###***21.	R1 -> L1 -> F2 -> T1 -> D2 -> FILTERING***

In [ ]:
prove(df.copy(), [1,1,2,1,2,1], 10)

Puntuación del clasificaodr KNN:  0.7168484848484848
Puntuación del clasificaodr Arból de Decisión:  0.7677575757575756
Puntuación del clasificaodr SVM:  0.6896363636363637


###***22.	R2 -> L1 -> F2 -> T1 -> D2 -> FILTERING***

In [ ]:
prove(df.copy(), [2,1,2,1,2,1], 10)

Puntuación del clasificaodr KNN:  0.7111637025112962
Puntuación del clasificaodr Arból de Decisión:  0.769081795151761
Puntuación del clasificaodr SVM:  0.6916090882542916


###***23.	R1 -> L2 -> F2 -> T1 -> D2 -> FILTERING***

In [ ]:
prove(df.copy(), [1,2,2,1,2,1], 10)

Puntuación del clasificaodr KNN:  0.7168484848484848
Puntuación del clasificaodr Arból de Decisión:  0.7677575757575756
Puntuación del clasificaodr SVM:  0.6896363636363637


###***24.	R2 -> L2 -> F2 -> T1 -> D2 -> FILTERING***

In [ ]:
prove(df.copy(), [2,2,2,1,2,1], 10)

Puntuación del clasificaodr KNN:  0.7816734494827653
Puntuación del clasificaodr Arból de Decisión:  0.7816734494827653
Puntuación del clasificaodr SVM:  0.7173365686574915


###***25.	R1 -> L1 -> F1 -> T2 -> D2 -> FILTERING***

In [ ]:
prove(df.copy(), [1,1,1,2,2,1], 10)

Puntuación del clasificaodr KNN:  0.7205454545454547
Puntuación del clasificaodr Arból de Decisión:  0.7680606060606061
Puntuación del clasificaodr SVM:  0.7049090909090909


###***26.	R2 -> L1 -> F1 -> T2 -> D2 -> FILTERING***

In [ ]:
prove(df.copy(), [2,1,1,2,2,1], 10)

Puntuación del clasificaodr KNN:  0.7081636550387765
Puntuación del clasificaodr Arból de Decisión:  0.7695014594203461
Puntuación del clasificaodr SVM:  0.706721461463103


###***27.	R1 -> L2 -> F1 -> T2 -> D2 -> FILTERING***

In [ ]:
prove(df.copy(), [1,2,1,2,2,1], 10)

Puntuación del clasificaodr KNN:  0.7205454545454547
Puntuación del clasificaodr Arból de Decisión:  0.7680606060606061
Puntuación del clasificaodr SVM:  0.7049090909090909


###***28.	R2 -> L2 -> F1 -> T2 -> D2 -> FILTERING***

In [ ]:
prove(df.copy(), [2,2,1,2,2,1], 10)

Puntuación del clasificaodr KNN:  0.7816734494827653
Puntuación del clasificaodr Arból de Decisión:  0.7816734494827653
Puntuación del clasificaodr SVM:  0.7173365686574915


###***29.	R1 -> L1 -> F2 -> T2 -> D2 -> FILTERING***

In [ ]:
prove(df.copy(), [1,1,2,2,2,1], 10)

Puntuación del clasificaodr KNN:  0.7205454545454547
Puntuación del clasificaodr Arból de Decisión:  0.7680606060606061
Puntuación del clasificaodr SVM:  0.7049090909090909


###***30.	R2 -> L1 -> F2 -> T2 -> D2 -> FILTERING***

In [ ]:
prove(df.copy(), [2,1,2,2,2,1], 10)

Puntuación del clasificaodr KNN:  0.7081636550387765
Puntuación del clasificaodr Arból de Decisión:  0.7695014594203461
Puntuación del clasificaodr SVM:  0.706721461463103


###***31.	R1 -> L2 -> F2 -> T2 -> D2 -> FILTERING***

In [ ]:
prove(df.copy(), [1,2,2,2,2,1], 10)

Puntuación del clasificaodr KNN:  0.7205454545454547
Puntuación del clasificaodr Arból de Decisión:  0.7680606060606061
Puntuación del clasificaodr SVM:  0.7049090909090909


###***32.	R2 -> L2 -> F2 -> T2 -> D2 -> FILTERING***

In [ ]:
prove(df.copy(), [2,2,2,2,2,1], 10)

Puntuación del clasificaodr KNN:  0.7816734494827653
Puntuación del clasificaodr Arból de Decisión:  0.7816734494827653
Puntuación del clasificaodr SVM:  0.7173365686574915


###***33. R1 -> L1 -> F1 -> T1 -> D1 -> EMBEDDED***

In [ ]:
prove(df.copy(), [1,1,1,1,1,2], 10)

Puntuación del clasificaodr KNN:  0.5729090909090908
Puntuación del clasificaodr Arból de Decisión:  0.7300000000000001
Puntuación del clasificaodr SVM:  0.7887878787878787


###***34. R2 -> L1 -> F1 -> T1 -> D1 -> EMBEDDED***

In [ ]:
prove(df.copy(), [2,1,1,1,1,2], 10)

Puntuación del clasificaodr KNN:  0.6098293290982092
Puntuación del clasificaodr Arból de Decisión:  0.739639410247447
Puntuación del clasificaodr SVM:  0.7933678372239222


###***35.	R1 -> L2 -> F1 -> T1 -> D1 -> EMBEDDED***

In [ ]:
prove(df.copy(), [1,2,1,1,1,2], 10)

Puntuación del clasificaodr KNN:  0.5729090909090908
Puntuación del clasificaodr Arból de Decisión:  0.7272727272727273
Puntuación del clasificaodr SVM:  0.7887878787878787


###***36.	R2 -> L2 -> F1 -> T1 -> D1 -> EMBEDDED***

In [ ]:
prove(df.copy(), [2,2,1,1,1,2], 10)

Puntuación del clasificaodr KNN:  0.6083898328248019
Puntuación del clasificaodr Arból de Decisión:  0.7423973119045255
Puntuación del clasificaodr SVM:  0.7949838809216575


###***37.	R1 -> L1 -> F2 -> T1 -> D1 -> EMBEDDED***

In [ ]:
prove(df.copy(), [1,1,2,1,1,2], 10)

Puntuación del clasificaodr KNN:  0.5572727272727274
Puntuación del clasificaodr Arból de Decisión:  0.6867272727272729
Puntuación del clasificaodr SVM:  0.715030303030303


###***38.	R2 -> L1 -> F2 -> T1 -> D1 -> EMBEDDED***

In [ ]:
prove(df.copy(), [2,1,2,1,1,2], 10)

Puntuación del clasificaodr KNN:  0.5810524585730337
Puntuación del clasificaodr Arból de Decisión:  0.6757317960868257
Puntuación del clasificaodr SVM:  0.7089927338273352


###***39.	R1 -> L2 -> F2 -> T1 -> D1 -> EMBEDDED***

In [ ]:
prove(df.copy(), [1,2,2,1,1,2], 10)

Puntuación del clasificaodr KNN:  0.5572727272727274
Puntuación del clasificaodr Arból de Decisión:  0.6811515151515151
Puntuación del clasificaodr SVM:  0.715030303030303


###***40.	R2 -> L2 -> F2 -> T1 -> D1 -> EMBEDDED***

In [ ]:
prove(df.copy(), [2,2,2,1,1,2], 10)

Puntuación del clasificaodr KNN:  0.577574736851191
Puntuación del clasificaodr Arból de Decisión:  0.6771743133387711
Puntuación del clasificaodr SVM:  0.7104938724485319


###***41.	R1 -> L1 -> F1 -> T2 -> D1 -> EMBEDDED***

In [ ]:
prove(df.copy(), [1,1,1,2,1,2], 10)

Puntuación del clasificaodr KNN:  0.6595757575757576
Puntuación del clasificaodr Arból de Decisión:  0.7708484848484848
Puntuación del clasificaodr SVM:  0.8064848484848485


###***42.	R2 -> L1 -> F1 -> T2 -> D1 -> EMBEDDED***

In [ ]:
prove(df.copy(), [2,1,1,2,1,2], 10)

Puntuación del clasificaodr KNN:  0.6929884167051481
Puntuación del clasificaodr Arból de Decisión:  0.7708175271420536
Puntuación del clasificaodr SVM:  0.8021169147465471


###***43.	R1 -> L2 -> F1 -> T2 -> D1 -> EMBEDDED***

In [ ]:
prove(df.copy(), [1,2,1,2,1,2], 10)

Puntuación del clasificaodr KNN:  0.6595757575757576
Puntuación del clasificaodr Arból de Decisión:  0.7683030303030304
Puntuación del clasificaodr SVM:  0.8064848484848485


###***44.	R2 -> L2 -> F1 -> T2 -> D1 -> EMBEDDED***



In [ ]:
prove(df.copy(), [2,2,1,2,1,2], 10)

Puntuación del clasificaodr KNN:  0.694010226731632
Puntuación del clasificaodr Arból de Decisión:  0.7697384623794666
Puntuación del clasificaodr SVM:  0.7950405961973072


###***45.	R1 -> L1 -> F2 -> T2 -> D1 -> EMBEDDED***

In [ ]:
prove(df.copy(), [1,1,2,2,1,2], 10)

Puntuación del clasificaodr KNN:  0.6573939393939394
Puntuación del clasificaodr Arból de Decisión:  0.7313333333333334
Puntuación del clasificaodr SVM:  0.7547878787878788


###***46.	R2 -> L1 -> F2 -> T2 -> D1 -> EMBEDDED***

In [ ]:
prove(df.copy(), [2,1,2,2,1,2], 10)

Puntuación del clasificaodr KNN:  0.6864564137532205
Puntuación del clasificaodr Arból de Decisión:  0.7192589539646027
Puntuación del clasificaodr SVM:  0.7464654191463866


###***47.	R1 -> L2 -> F2 -> T2 -> D1 -> EMBEDDED***

In [ ]:
prove(df.copy(), [1,2,2,2,1,2], 10)

Puntuación del clasificaodr KNN:  0.6573939393939394
Puntuación del clasificaodr Arból de Decisión:  0.7247272727272727
Puntuación del clasificaodr SVM:  0.7547878787878788


###***48.	R2 -> L2 -> F2 -> T2 -> D1 -> EMBEDDED***


In [ ]:
prove(df.copy(), [2,2,2,2,1,2], 10)

Puntuación del clasificaodr KNN:  0.6741042438994216
Puntuación del clasificaodr Arból de Decisión:  0.7190791697775554
Puntuación del clasificaodr SVM:  0.7393888848129656


###***49.	R1 -> L1 -> F1 -> T1 -> D2 -> EMBEDDED***

In [ ]:
prove(df.copy(), [1,1,1,1,2,2], 10)

Puntuación del clasificaodr KNN:  0.5012727272727273
Puntuación del clasificaodr Arból de Decisión:  0.6510909090909089
Puntuación del clasificaodr SVM:  0.7036363636363635


###***50.	R2 -> L1 -> F1 -> T1 -> D2 -> EMBEDDED***

In [ ]:
prove(df.copy(), [2,1,1,1,2,2], 10)

Puntuación del clasificaodr KNN:  0.5216882163135719
Puntuación del clasificaodr Arból de Decisión:  0.6565316073475953
Puntuación del clasificaodr SVM:  0.7058118592108917


###***51.	R1 -> L2 -> F1 -> T1 -> D2 -> EMBEDDED***

In [ ]:
prove(df.copy(), [1,2,1,1,2,2], 10)

Puntuación del clasificaodr KNN:  0.5012727272727273
Puntuación del clasificaodr Arból de Decisión:  0.6503636363636364
Puntuación del clasificaodr SVM:  0.7036363636363635


###***52.	R2 -> L2 -> F1 -> T1 -> D2 -> EMBEDDED***

In [ ]:
prove(df.copy(), [1,2,1,1,2,2], 10)

Puntuación del clasificaodr KNN:  0.5012727272727273
Puntuación del clasificaodr Arból de Decisión:  0.6476969696969697
Puntuación del clasificaodr SVM:  0.7036363636363635


###***53.	R1 -> L1 -> F2 -> T1 -> D2 -> EMBEDDED***

In [ ]:
prove(df.copy(), [1,1,2,1,2,2], 10)

Puntuación del clasificaodr KNN:  0.4787878787878788
Puntuación del clasificaodr Arból de Decisión:  0.5902424242424242
Puntuación del clasificaodr SVM:  0.6994545454545456


###***54.	R2 -> L1 -> F2 -> T1 -> D2 -> EMBEDDED***

In [ ]:
prove(df.copy(), [2,1,2,1,2,2], 10)

Puntuación del clasificaodr KNN:  0.49675086565420734
Puntuación del clasificaodr Arból de Decisión:  0.5901696279449146
Puntuación del clasificaodr SVM:  0.6904046169183429


###***55.	R1 -> L2 -> F2 -> T1 -> D2 -> EMBEDDED***

In [ ]:
prove(df.copy(), [1,2,2,1,2,2], 10)

Puntuación del clasificaodr KNN:  0.4787878787878788
Puntuación del clasificaodr Arból de Decisión:  0.5942424242424243
Puntuación del clasificaodr SVM:  0.6994545454545456


###***56.	R2 -> L2 -> F2 -> T1 -> D2 -> EMBEDDED***

In [ ]:
prove(df.copy(), [2,2,2,1,2,2], 10)

Puntuación del clasificaodr KNN:  0.4890761056421809
Puntuación del clasificaodr Arból de Decisión:  0.5997040160313262
Puntuación del clasificaodr SVM:  0.6872873267073203


###***57.	R1 -> L1 -> F1 -> T2 -> D2 -> EMBEDDED***

In [ ]:
prove(df.copy(), [1,1,1,2,2,2], 10)

Puntuación del clasificaodr KNN:  0.5848484848484847
Puntuación del clasificaodr Arból de Decisión:  0.692969696969697
Puntuación del clasificaodr SVM:  0.708


###***58.	R2 -> L1 -> F1 -> T2 -> D2 -> EMBEDDED***

In [ ]:
prove(df.copy(), [2,1,1,2,2,2], 10)

Puntuación del clasificaodr KNN:  0.6019691025823612
Puntuación del clasificaodr Arból de Decisión:  0.6914244848871952
Puntuación del clasificaodr SVM:  0.7091684181149382


###***59.	R1 -> L2 -> F1 -> T2 -> D2 -> EMBEDDED***

In [ ]:
prove(df.copy(), [1,2,1,2,2,2], 10)

Puntuación del clasificaodr KNN:  0.5848484848484847
Puntuación del clasificaodr Arból de Decisión:  0.7026060606060606
Puntuación del clasificaodr SVM:  0.708


###***60.	R2 -> L2 -> F1 -> T2 -> D2 -> EMBEDDED***

In [ ]:
prove(df.copy(), [2,2,1,2,2,2], 10)

Puntuación del clasificaodr KNN:  0.6024528187887603
Puntuación del clasificaodr Arból de Decisión:  0.6904048686665544
Puntuación del clasificaodr SVM:  0.7070694494194687


###***61.	R1 -> L1 -> F2 -> T2 -> D2 -> EMBEDDED***

In [ ]:
prove(df.copy(), [1,1,2,2,2,2], 10)

Puntuación del clasificaodr KNN:  0.5834545454545454
Puntuación del clasificaodr Arból de Decisión:  0.6311515151515151
Puntuación del clasificaodr SVM:  0.712969696969697


###***62.	R2 -> L1 -> F2 -> T2 -> D2 -> EMBEDDED***

In [ ]:
prove(df.copy(), [2,1,2,2,2,2], 10)

Puntuación del clasificaodr KNN:  0.6001140059757832
Puntuación del clasificaodr Arból de Decisión:  0.6237984417504989
Puntuación del clasificaodr SVM:  0.7113278783092303


###***63. R1 -> L2 -> F2 -> T2 -> D2 -> EMBEDDED***

In [ ]:
prove(df.copy(), [1,2,2,2,2,2], 10)

Puntuación del clasificaodr KNN:  0.5834545454545454
Puntuación del clasificaodr Arból de Decisión:  0.6283030303030303
Puntuación del clasificaodr SVM:  0.712969696969697


###***64.	R2 -> L2 -> F2 -> T2 -> D2 -> EMBEDDED***

In [ ]:
prove(df.copy(), [2,2,2,2,2,2], 10)

Puntuación del clasificaodr KNN:  0.5886038259973904
Puntuación del clasificaodr Arból de Decisión:  0.6260761876401699
Puntuación del clasificaodr SVM:  0.6971756727791132


###***65. R1 -> L1 -> F1 -> T1 -> D1 -> WRAPPER***

In [ ]:
prove(df.copy(), [1,1,1,1,1,3], 10)

Puntuación del clasificaodr KNN:  0.5729090909090908
Puntuación del clasificaodr Arból de Decisión:  0.7306666666666667
Puntuación del clasificaodr SVM:  0.666848484848485


###***66. R2 -> L1 -> F1 -> T1 -> D1 -> WRAPPER***

In [ ]:
prove(df.copy(), [2,1,1,1,1,3], 10)

Puntuación del clasificaodr KNN:  0.6098293290982092
Puntuación del clasificaodr Arból de Decisión:  0.7417986546575578
Puntuación del clasificaodr SVM:  0.7933678372239222


###***67.	R1 -> L2 -> F1 -> T1 -> D1 -> WRAPPER***

In [ ]:
prove(df.copy(), [1,2,1,1,1,3], 10)

Puntuación del clasificaodr KNN:  0.5729090909090908
Puntuación del clasificaodr Arból de Decisión:  0.7264242424242424
Puntuación del clasificaodr SVM:  0.666848484848485


###***68.	R2 -> L2 -> F1 -> T1 -> D1 -> WRAPPER***

In [ ]:
prove(df.copy(), [2,2,1,1,1,3], 10)

Puntuación del clasificaodr KNN:  0.6083898328248019
Puntuación del clasificaodr Arból de Decisión:  0.737900117818163
Puntuación del clasificaodr SVM:  0.7949838809216575


###***69.	R1 -> L1 -> F2 -> T1 -> D1 -> WRAPPER***

In [ ]:
prove(df.copy(), [1,1,2,1,1,3], 10)

Puntuación del clasificaodr KNN:  0.5572727272727274
Puntuación del clasificaodr Arból de Decisión:  0.6933333333333332
Puntuación del clasificaodr SVM:  0.715030303030303


###***70.	R2 -> L1 -> F2 -> T1 -> D1 -> WRAPPER***

In [ ]:
prove(df.copy(), [2,1,2,1,1,3], 10)

Puntuación del clasificaodr KNN:  0.5462213672373439
Puntuación del clasificaodr Arból de Decisión:  0.5500621458442124
Puntuación del clasificaodr SVM:  0.7578541845587717


###***71.	R1 -> L2 -> F2 -> T1 -> D1 -> WRAPPER***

In [ ]:
prove(df.copy(), [1,2,2,1,1,3], 10)

Puntuación del clasificaodr KNN:  0.5572727272727274
Puntuación del clasificaodr Arból de Decisión:  0.6875757575757577
Puntuación del clasificaodr SVM:  0.715030303030303


###***72.	R2 -> L2 -> F2 -> T1 -> D1 -> WRAPPER***

In [ ]:
prove(df.copy(), [2,2,2,1,1,3], 10)

Puntuación del clasificaodr KNN:  0.6659484649832624
Puntuación del clasificaodr Arból de Decisión:  0.7194398170725567
Puntuación del clasificaodr SVM:  0.7524585011055342


###***73.	R1 -> L1 -> F1 -> T2 -> D1 -> WRAPPER***

In [ ]:
prove(df.copy(), [1,1,1,2,1,3], 10)

Puntuación del clasificaodr KNN:  0.6022424242424242
Puntuación del clasificaodr Arból de Decisión:  0.7298787878787878
Puntuación del clasificaodr SVM:  0.7762424242424242


###***74.	R2 -> L1 -> F1 -> T2 -> D1 -> WRAPPER***

In [ ]:
prove(df.copy(), [2,1,1,2,1,3], 10)

Puntuación del clasificaodr KNN:  0.625303176774717
Puntuación del clasificaodr Arból de Decisión:  0.7121875265234723
Puntuación del clasificaodr SVM:  0.7299732859183558


###***75.	R1 -> L2 -> F1 -> T2 -> D1 -> WRAPPER***

In [ ]:
prove(df.copy(), [1,2,1,2,1,3], 10)

Puntuación del clasificaodr KNN:  0.6022424242424242
Puntuación del clasificaodr Arból de Decisión:  0.729090909090909
Puntuación del clasificaodr SVM:  0.7762424242424242


###***76.	R2 -> L2 -> F1 -> T2 -> D1 -> WRAPPER***



In [ ]:
prove(df.copy(), [2,2,1,2,1,3], 10)

Puntuación del clasificaodr KNN:  0.6609732010432446
Puntuación del clasificaodr Arból de Decisión:  0.719975465338587
Puntuación del clasificaodr SVM:  0.7427471699904624


###***77.	R1 -> L1 -> F2 -> T2 -> D1 -> WRAPPER***

In [ ]:
prove(df.copy(), [1,1,2,2,1,3], 10)

Puntuación del clasificaodr KNN:  0.5944242424242424
Puntuación del clasificaodr Arból de Decisión:  0.6870303030303029
Puntuación del clasificaodr SVM:  0.7069090909090909


###***78.	R2 -> L1 -> F2 -> T2 -> D1 -> WRAPPER***

In [ ]:
prove(df.copy(), [2,1,2,2,1,3], 10)

Puntuación del clasificaodr KNN:  0.6026331784002912
Puntuación del clasificaodr Arból de Decisión:  0.6934105984558483
Puntuación del clasificaodr SVM:  0.7305142208968279


###***79.	R1 -> L2 -> F2 -> T2 -> D1 -> WRAPPER***

In [ ]:
prove(df.copy(), [1,2,2,2,1,3], 10)

Puntuación del clasificaodr KNN:  0.5944242424242424
Puntuación del clasificaodr Arból de Decisión:  0.6832727272727273
Puntuación del clasificaodr SVM:  0.7069090909090909


###***80.	R2 -> L2 -> F2 -> T2 -> D1 -> WRAPPER***


In [ ]:
prove(df.copy(), [2,2,2,2,1,3], 10)

Puntuación del clasificaodr KNN:  0.6741042438994216
Puntuación del clasificaodr Arból de Decisión:  0.7249556203867141
Puntuación del clasificaodr SVM:  0.7393888848129656


###***81.	R1 -> L1 -> F1 -> T1 -> D2 -> WRAPPER***

In [ ]:
prove(df.copy(), [1,1,1,1,2,3], 10)

Puntuación del clasificaodr KNN:  0.5012727272727273
Puntuación del clasificaodr Arból de Decisión:  0.6518787878787878
Puntuación del clasificaodr SVM:  0.666848484848485


###***82.	R2 -> L1 -> F1 -> T1 -> D2 -> WRAPPER***

In [ ]:
prove(df.copy(), [2,1,1,1,2,3], 10)

Puntuación del clasificaodr KNN:  0.5216882163135719
Puntuación del clasificaodr Arból de Decisión:  0.6564701448199568
Puntuación del clasificaodr SVM:  0.7058118592108917


###***83.	R1 -> L2 -> F1 -> T1 -> D2 -> WRAPPER***

In [ ]:
prove(df.copy(), [1,2,1,1,2,3], 10)

Puntuación del clasificaodr KNN:  0.5012727272727273
Puntuación del clasificaodr Arból de Decisión:  0.6519393939393939
Puntuación del clasificaodr SVM:  0.666848484848485


###***84.	R2 -> L2 -> F1 -> T1 -> D2 -> WRAPPER***

In [ ]:
prove(df.copy(), [1,2,1,1,2,3], 10)

Puntuación del clasificaodr KNN:  0.5012727272727273
Puntuación del clasificaodr Arból de Decisión:  0.6528484848484849
Puntuación del clasificaodr SVM:  0.666848484848485


###***85.	R1 -> L1 -> F2 -> T1 -> D2 -> WRAPPER***

In [ ]:
prove(df.copy(), [1,1,2,1,2,3], 10)

Puntuación del clasificaodr KNN:  0.4787878787878788
Puntuación del clasificaodr Arból de Decisión:  0.5982424242424244
Puntuación del clasificaodr SVM:  0.6994545454545456


###***86.	R2 -> L1 -> F2 -> T1 -> D2 -> WRAPPER***

In [ ]:
prove(df.copy(), [2,1,2,1,2,3], 10)

Puntuación del clasificaodr KNN:  0.4704906141073943
Puntuación del clasificaodr Arból de Decisión:  0.4315870998462178
Puntuación del clasificaodr SVM:  0.7534184170360174


###***87.	R1 -> L2 -> F2 -> T1 -> D2 -> WRAPPER***

In [ ]:
prove(df.copy(), [1,2,2,1,2,3], 10)

Puntuación del clasificaodr KNN:  0.4787878787878788
Puntuación del clasificaodr Arból de Decisión:  0.5933333333333334
Puntuación del clasificaodr SVM:  0.6994545454545456


###***88.	R2 -> L2 -> F2 -> T1 -> D2 -> WRAPPER***

In [ ]:
prove(df.copy(), [2,2,2,1,2,3], 10)

Puntuación del clasificaodr KNN:  0.5795470761962715
Puntuación del clasificaodr Arból de Decisión:  0.624877038980693
Puntuación del clasificaodr SVM:  0.7272130465993132


###***89.	R1 -> L1 -> F1 -> T2 -> D2 -> WRAPPER***

In [ ]:
prove(df.copy(), [1,1,1,2,2,3], 10)

Puntuación del clasificaodr KNN:  0.5336969696969696
Puntuación del clasificaodr Arból de Decisión:  0.6487272727272728
Puntuación del clasificaodr SVM:  0.6844242424242424


###***90.	R2 -> L1 -> F1 -> T2 -> D2 -> WRAPPER***

In [ ]:
prove(df.copy(), [2,1,1,2,2,3], 10)

Puntuación del clasificaodr KNN:  0.5462728317645823
Puntuación del clasificaodr Arból de Decisión:  0.6380150228947016
Puntuación del clasificaodr SVM:  0.7184045924628024


###***91.	R1 -> L2 -> F1 -> T2 -> D2 -> WRAPPER***

In [ ]:
prove(df.copy(), [1,2,1,2,2,3], 10)

Puntuación del clasificaodr KNN:  0.5336969696969696
Puntuación del clasificaodr Arból de Decisión:  0.6498181818181817
Puntuación del clasificaodr SVM:  0.6844242424242424


###***92.	R2 -> L2 -> F1 -> T2 -> D2 -> WRAPPER***

In [ ]:
prove(df.copy(), [2,2,1,2,2,3], 10)

Puntuación del clasificaodr KNN:  0.5757100738125757
Puntuación del clasificaodr Arból de Decisión:  0.6433458991654907
Puntuación del clasificaodr SVM:  0.7057565105683898


###***93.	R1 -> L1 -> F2 -> T2 -> D2 -> WRAPPER***

In [ ]:
prove(df.copy(), [1,1,2,2,2,3], 10)

Puntuación del clasificaodr KNN:  0.5274545454545454
Puntuación del clasificaodr Arból de Decisión:  0.5911515151515151
Puntuación del clasificaodr SVM:  0.6754545454545455


###***94.	R2 -> L1 -> F2 -> T2 -> D2 -> WRAPPER***

In [ ]:
prove(df.copy(), [2,1,2,2,2,3], 10)

Puntuación del clasificaodr KNN:  0.5323597510713685
Puntuación del clasificaodr Arból de Decisión:  0.6004713445800048
Puntuación del clasificaodr SVM:  0.7178641609807535


###***95. R1 -> L2 -> F2 -> T2 -> D2 -> WRAPPER***

In [ ]:
prove(df.copy(), [1,2,2,2,2,3], 10)

Puntuación del clasificaodr KNN:  0.5274545454545454
Puntuación del clasificaodr Arból de Decisión:  0.5918787878787878
Puntuación del clasificaodr SVM:  0.6754545454545455


###***96.	R2 -> L2 -> F2 -> T2 -> D2 -> WRAPPER***

In [ ]:
prove(df.copy(), [2,2,2,2,2,3], 10)

Puntuación del clasificaodr KNN:  0.5886038259973904
Puntuación del clasificaodr Arból de Decisión:  0.6271560076473914
Puntuación del clasificaodr SVM:  0.6971756727791132


###***97. R1 -> L1 -> F1 -> T1 -> D1 -> PCA***

In [ ]:
prove(df.copy(), [1,1,1,1,1,4], 10)

Puntuación del clasificaodr KNN:  0.6799153119016484
Puntuación del clasificaodr Arból de Decisión:  0.6335706599687256
Puntuación del clasificaodr SVM:  0.7634900358057886


###***98. R2 -> L1 -> F1 -> T1 -> D1 -> PCA***

In [ ]:
prove(df.copy(), [2,1,1,1,1,4], 10)

Puntuación del clasificaodr KNN:  0.6799153119016484
Puntuación del clasificaodr Arból de Decisión:  0.6326704802924308
Puntuación del clasificaodr SVM:  0.7634900358057886


###***99.	R1 -> L2 -> F1 -> T1 -> D1 -> PCA***

In [ ]:
prove(df.copy(), [1,2,1,1,1,4], 10)

Puntuación del clasificaodr KNN:  0.6929874097123021
Puntuación del clasificaodr Arból de Decisión:  0.6420226746017702
Puntuación del clasificaodr SVM:  0.7686456593573373


###***100.	R2 -> L2 -> F1 -> T1 -> D1 -> PCA***

In [ ]:
prove(df.copy(), [2,2,1,1,1,4], 10)

Puntuación del clasificaodr KNN:  0.6929874097123021
Puntuación del clasificaodr Arból de Decisión:  0.6389056361389593
Puntuación del clasificaodr SVM:  0.7686456593573373


###***101.	R1 -> L1 -> F2 -> T1 -> D1 -> PCA***



In [ ]:
prove(df.copy(), [1,1,2,1,1,4], 10)

Puntuación del clasificaodr KNN:  0.6193066422686686
Puntuación del clasificaodr Arból de Decisión:  0.6196067980648475
Puntuación del clasificaodr SVM:  0.7051592199545702


###***102.	R2 -> L1 -> F2 -> T1 -> D1 -> PCA***

In [ ]:
prove(df.copy(), [2,1,2,1,1,4], 10)

Puntuación del clasificaodr KNN:  0.6799153119016484
Puntuación del clasificaodr Arból de Decisión:  0.6344694730118725
Puntuación del clasificaodr SVM:  0.7634900358057886


###***103.	R1 -> L2 -> F2 -> T1 -> D1 -> PCA***

In [ ]:
prove(df.copy(), [1,2,2,1,1,4], 10)

Puntuación del clasificaodr KNN:  0.6929874097123021
Puntuación del clasificaodr Arból de Decisión:  0.6400444371557343
Puntuación del clasificaodr SVM:  0.7686456593573373


###***104.	R2 -> L2 -> F2 -> T1 -> D1 -> PCA***

In [ ]:
prove(df.copy(), [2,2,2,1,1,4], 10)

Puntuación del clasificaodr KNN:  0.6929874097123021
Puntuación del clasificaodr Arból de Decisión:  0.6381856722180744
Puntuación del clasificaodr SVM:  0.7686456593573373


###***105.	R1 -> L1 -> F1 -> T2 -> D1 -> PCA***

In [ ]:
prove(df.copy(), [1,1,1,2,1,4], 10)

Puntuación del clasificaodr KNN:  0.6799153119016484
Puntuación del clasificaodr Arból de Decisión:  0.6356691611318024
Puntuación del clasificaodr SVM:  0.7634900358057886


###***106.	R2 -> L1 -> F1 -> T2 -> D1 -> PCA***

In [ ]:
prove(df.copy(), [2,1,1,2,1,4], 10)

Puntuación del clasificaodr KNN:  0.6799153119016484
Puntuación del clasificaodr Arból de Decisión:  0.6324903364650811
Puntuación del clasificaodr SVM:  0.7634900358057886


###***107.	R1 -> L2 -> F1 -> T2 -> D1 -> PCA***

In [ ]:
prove(df.copy(), [1,2,1,2,1,4], 10)

Puntuación del clasificaodr KNN:  0.6929874097123021
Puntuación del clasificaodr Arból de Decisión:  0.6401646289447147
Puntuación del clasificaodr SVM:  0.7686456593573373


###***108.	R2 -> L2 -> F1 -> T2 -> D1 -> PCA***



In [ ]:
prove(df.copy(), [2,2,1,2,1,4], 10)

Puntuación del clasificaodr KNN:  0.6929874097123021
Puntuación del clasificaodr Arból de Decisión:  0.6399843412612441
Puntuación del clasificaodr SVM:  0.7686456593573373


###***109.	R1 -> L1 -> F2 -> T2 -> D1 -> PCA***

In [ ]:
prove(df.copy(), [1,1,2,2,1,4], 10)

Puntuación del clasificaodr KNN:  0.6193066422686686
Puntuación del clasificaodr Arból de Decisión:  0.6193069659449406
Puntuación del clasificaodr SVM:  0.7051592199545702


###***110.	R2 -> L1 -> F2 -> T2 -> D1 -> PCA***

In [ ]:
prove(df.copy(), [2,1,2,2,1,4], 10)

Puntuación del clasificaodr KNN:  0.6799153119016484
Puntuación del clasificaodr Arból de Decisión:  0.6326705162564609
Puntuación del clasificaodr SVM:  0.7634300478033891


###***111.	R1 -> L2 -> F2 -> T2 -> D1 -> PCA***

In [ ]:
prove(df.copy(), [1,2,2,2,1,4], 10)

Puntuación del clasificaodr KNN:  0.6929874097123021
Puntuación del clasificaodr Arból de Decisión:  0.6411837416689324
Puntuación del clasificaodr SVM:  0.7686456593573373


###***112.	R2 -> L2 -> F2 -> T2 -> D1 -> PCA***


In [ ]:
prove(df.copy(), [2,2,2,2,1,4], 10)

Puntuación del clasificaodr KNN:  0.6929874097123021
Puntuación del clasificaodr Arból de Decisión:  0.6396850126377602
Puntuación del clasificaodr SVM:  0.7686456593573373


###***113.	R1 -> L1 -> F1 -> T1 -> D2 -> PCA***

In [ ]:
prove(df.copy(), [1,1,1,1,2,4], 10)

Puntuación del clasificaodr KNN:  0.6746963916569205
Puntuación del clasificaodr Arból de Decisión:  0.5745070050738054
Puntuación del clasificaodr SVM:  0.7467032852422323


###***114.	R2 -> L1 -> F1 -> T1 -> D2 -> PCA***

In [ ]:
prove(df.copy(), [2,1,1,1,2,4], 10)

Puntuación del clasificaodr KNN:  0.6746963916569205
Puntuación del clasificaodr Arból de Decisión:  0.5761264653544111
Puntuación del clasificaodr SVM:  0.7467632372806016


###***115.	R1 -> L2 -> F1 -> T1 -> D2 ->PCA***

In [ ]:
prove(df.copy(), [1,2,1,1,2,4], 10)

Puntuación del clasificaodr KNN:  0.6819518470406638
Puntuación del clasificaodr Arból de Decisión:  0.5818232396686132
Puntuación del clasificaodr SVM:  0.749041414738635


###***116.	R2 -> L2 -> F1 -> T1 -> D2 -> PCA***

In [ ]:
prove(df.copy(), [1,2,1,1,2,4], 10)

Puntuación del clasificaodr KNN:  0.6819518470406638
Puntuación del clasificaodr Arból de Decisión:  0.58044452260627
Puntuación del clasificaodr SVM:  0.749041414738635


###***117.	R1 -> L1 -> F2 -> T1 -> D2 -> PCA***

In [ ]:
prove(df.copy(), [1,1,2,1,2,4], 10)

Puntuación del clasificaodr KNN:  0.6746963916569205
Puntuación del clasificaodr Arból de Decisión:  0.5736685396733604
Puntuación del clasificaodr SVM:  0.7467632372806016


###***118.	R2 -> L1 -> F2 -> T1 -> D2 -> PCA***

In [ ]:
prove(df.copy(), [2,1,2,1,2,4], 10)

Puntuación del clasificaodr KNN:  0.6746963916569205
Puntuación del clasificaodr Arból de Decisión:  0.5761264653544111
Puntuación del clasificaodr SVM:  0.7467032852422323


###***119.	R1 -> L2 -> F2 -> T1 -> D2 -> PCA***

In [ ]:
prove(df.copy(), [1,2,2,1,2,4], 10)

Puntuación del clasificaodr KNN:  0.6819518470406638
Puntuación del clasificaodr Arból de Decisión:  0.5781659495439041
Puntuación del clasificaodr SVM:  0.749041414738635


###***120.	R2 -> L2 -> F2 -> T1 -> D2 -> PCA***

In [ ]:
prove(df.copy(), [2,2,2,1,2,4], 10)

Puntuación del clasificaodr KNN:  0.6819518470406638
Puntuación del clasificaodr Arból de Decisión:  0.581463491474367
Puntuación del clasificaodr SVM:  0.749041414738635


###***121.	R1 -> L1 -> F1 -> T2 -> D2 -> PCA***

In [ ]:
prove(df.copy(), [1,1,1,2,2,4], 10)

Puntuación del clasificaodr KNN:  0.6746963916569205
Puntuación del clasificaodr Arból de Decisión:  0.5742079281985331
Puntuación del clasificaodr SVM:  0.7467632372806016


###***122.	R2 -> L1 -> F1 -> T2 -> D2 -> PCA***

In [ ]:
prove(df.copy(), [2,1,1,2,2,4], 10)

Puntuación del clasificaodr KNN:  0.6746963916569205
Puntuación del clasificaodr Arból de Decisión:  0.571809379131368
Puntuación del clasificaodr SVM:  0.7467032852422323


###***123.	R1 -> L2 -> F1 -> T2 -> D2 -> PCA***

In [ ]:
prove(df.copy(), [1,2,1,2,2,4], 10)

Puntuación del clasificaodr KNN:  0.6819518470406638
Puntuación del clasificaodr Arból de Decisión:  0.5803842468916288
Puntuación del clasificaodr SVM:  0.749041414738635


###***124.	R2 -> L2 -> F1 -> T2 -> D2 -> PCA***

In [ ]:
prove(df.copy(), [2,2,1,2,2,4], 10)

Puntuación del clasificaodr KNN:  0.6819518470406638
Puntuación del clasificaodr Arból de Decisión:  0.5820627241458183
Puntuación del clasificaodr SVM:  0.749041414738635


###***125.	R1 -> L1 -> F2 -> T2 -> D2 -> PCA***

In [ ]:
prove(df.copy(), [1,1,2,2,2,4], 10)

Puntuación del clasificaodr KNN:  0.6746963916569205
Puntuación del clasificaodr Arból de Decisión:  0.5745674246445674
Puntuación del clasificaodr SVM:  0.7467632372806016


###***126.	R2 -> L1 -> F2 -> T2 -> D2 -> PCA***

In [ ]:
prove(df.copy(), [2,1,2,2,2,4], 10)

Puntuación del clasificaodr KNN:  0.6746963916569205
Puntuación del clasificaodr Arból de Decisión:  0.5754668850402582
Puntuación del clasificaodr SVM:  0.7467632372806016


###***127. R1 -> L2 -> F2 -> T2 -> D2 -> PCA***

In [ ]:
prove(df.copy(), [1,2,2,2,2,4], 10)

Puntuación del clasificaodr KNN:  0.6819518470406638
Puntuación del clasificaodr Arból de Decisión:  0.5779256738580341
Puntuación del clasificaodr SVM:  0.749041414738635


###***128.	R2 -> L2 -> F2 -> T2 -> D2 -> PCA***

In [ ]:
prove(df.copy(), [2,2,2,2,2,4], 10)

Puntuación del clasificaodr KNN:  0.6819518470406638
Puntuación del clasificaodr Arból de Decisión:  0.5800844147717219
Puntuación del clasificaodr SVM:  0.749041414738635


## ***3. Comparación y Conclusión***

En esta parte, se realizará una comparación a nivel de todos los ***worflows*** realizados. Se seleccionaron los 5 mejores ***flujos de trabajo o workflows*** por cada clasificador. En este caso son ***3*** clasificadores. A continuación, se presentan 3 tablas. En cada tabla, se especifican las ***5*** combinaciones de preprocesamiento que determinan las mejores precisiones para respectivo clasificador. 

* ***Clasificador K Vecinos más Cercanos (KNN)***

![alt text](https://i.ibb.co/tHknfVC/knn.png)

* ***Clasificador Arbol de Decisión***

![alt text](https://i.ibb.co/hfbJLdY/ad.png)

* ***Clasificador Maquinas de Vectores de Soporte (SVM)***

![alt text](https://i.ibb.co/qy73Szw/maquinas.png)

Ahora bien, dadas estas tablas, es posible realizar ciertas conclusiones. En primer lugar, para el clasificador ***KNN***, Una de las mejores precisiones obtenidas es ***78.1673%***. Ello es posible a partir del siguiente esquema de preprocesamiento de datos:

* ***No reducir datos -> Limpieza con 0 -> Contruir caracteristicas de fecha -> Normalización -> Discretizar a 3 intervalos -> Filtering***

En segundo lugar, para el clasificador ***Arbol de Decisión***, Una de las mejores precisiones obtenidas es ***79.9478%***. Ello es posible a partir del siguiente esquema de preprocesamiento de datos:

* ***No reducir datos -> Limpieza con cero -> Construir Caracteristicas -> Normalización -> Discretizar a 2 intervalos -> Filtering***

Por ultimo, para el clasificador ***SVM***, Una de las mejores precisiones obtenidas es ***80.6485%***. Ello es posible a partir del siguiente esquema de preprocesamiento de datos:

* ***Reducir datos -> Limpieza con media -> Construir caracteristicas -> Estandarización -> Discretizar a 2 intervalos -> Embedded***

En general, la mejor precisión obtenida, a partir de todos los ***flujos de trabajo realizados*** y comparados, es ***80.6485%***. Se pudo alcanzar dicha precisión por medio de el ***clasificador SVM*** y las ***tecnicas de preprocesamiento de datos*** que se especifican a continuación:

* ***Reducir datos -> Limpieza con media -> Construir caracteristicas -> Estandarización -> Discretizar a 2 intervalos -> Embedded***

En sintesís, se ha logrado determinar ***128 flujos de trabajo***. Cada flujo de trabajo ha sido probado con 3 clasificadores. Se ha determinado el mejor ***flujo de trabajo*** para cada clasificador. Asimismo, se ha determinado la mejor precisión obtenida, a partir de los ***clasificadores y el preprocesamiento*** realizado, y el esquema de trabajo a seguir para llegar a dicha exactitud con respecto a la clasificación. En ese sentido, al
tener un gran set de ***workflows***, se logra verificar la variabilidad del clasificador frente al ***dataset*** y el ***pre-procesamiento realizado***.





